# Datafile Collation and Cleaning
#### NOTE : this file ONLY loads and collates the data. The remaining tasks necessary for full Data Cleaning and exploration are in EDA-post-processing because othis process takes hours.

#### The next notebook to look at is EDS_post_processing

In [1]:
import os
import pandas
import time

#turn off the really annoying deprecation messages
import warnings
warnings.filterwarnings('ignore')

#### Directories and filenames are declared at the top so changing them is a simple matter.

In [2]:
input_path = "./NACJD/"
output_path = "./output data/"
output_filename = "combined_and_cleaned.csv"
output_bias_filename = "bias_data.csv"

#### Scan all subdirectories of "input_path", returning a set of the names of files to be included.
- We use a set to eliminate duplicates. There should be none, but just to be sure.

In [3]:
set_of_files_to_process = set()
def get_list_of_files_to_process( input_folder_path = './', filename_regexp=".*csv" ) :
    """
    return a list of the files to be processed

    """
    #compile regexp for faster execution. also without pre-compiling it, there 
    #    are problems getting it to work
    filename_check = regex.compile( filename_regexp )
    
    #Verify that all parameters are of the correct type
    if type( input_folder_path ) != str :
        raise TypeError

    #recurse through all subdirs collecting full path and filenames that match filename_regexp
    
    #dirpath returns the full path from the starting directory to the current
    #dirnames returns a list of dirpath's subdirs
    #filenames returns a list of all files in dirpath
    for dirpath, dirnames, filenames in os.walk(input_folder_path, topdown=True):
        for folder in dirnames: # for each directory, call this function again until we 
                                #    reach the final leaf nodes
            get_list_of_files_to_process(input_folder_path = (dirpath + "/" + folder ), 
                                         filename_regexp = filename_regexp )
            
        #in each leaf node collect the filenames that match the regular expression argument
        for file in filenames :
            if filename_check.match( file ):
                full_path_and_filename = dirpath+"/"+file
                set_of_files_to_process.add( full_path_and_filename.replace( "\\", "/") )
            
    return set_of_files_to_process

In [4]:
set_of_files_to_process = get_list_of_files_to_process( input_folder_path = input_path,
                                                       filename_regexp=".*-0002-Data.tsv" )

#### Next collect the contents of all the files into a single dataframe

In [5]:
combined_data_table = pandas.DataFrame()
for current_file in set_of_files_to_process :
    datafile = None
    datafile = pandas.DataFrame()
    datafile = pandas.read_csv( current_file, sep="\t" )
    combined_data_table = pandas.concat( [combined_data_table, datafile], axis="rows" )
    print( "adding file ", current_file, " to bring size to ", combined_data_table.shape )


adding file  ./NACJD//1999 ICPSR_23800/DS0002/23800-0002-Data.tsv  to bring size to  (7875, 179)
adding file  ./NACJD/1996 ICPSR_23841/DS0002/23841-0002-Data.tsv  to bring size to  (16634, 179)
adding file  ./NACJD//2016 ICPSR_37060/DS0002/37060-0002-Data.tsv  to bring size to  (22902, 179)
adding file  ./NACJD//2011 ICPSR_34583/DS0002/34583-0002-Data.tsv  to bring size to  (29124, 179)
adding file  ./NACJD//2012 ICPSR_35086/DS0002/35086-0002-Data.tsv  to bring size to  (35717, 179)
adding file  ./NACJD/2006 ICPSR_22406/DS0002/22406-0002-Data.tsv  to bring size to  (43443, 179)
adding file  ./NACJD//2006 ICPSR_22406/DS0002/22406-0002-Data.tsv  to bring size to  (51169, 179)
adding file  ./NACJD//2014 ICPSR_36397/DS0002/36397-0002-Data.tsv  to bring size to  (56648, 179)
adding file  ./NACJD/1997 ICPSR_23840/DS0002/23840-0002-Data.tsv  to bring size to  (64696, 179)
adding file  ./NACJD/2011 ICPSR_34583/DS0002/34583-0002-Data.tsv  to bring size to  (70918, 179)
adding file  ./NACJD//199

#### See document ./FBI/ICPSR_33524/33524-Codebook.pdf for a data dictionary

In [6]:
column_names = combined_data_table.columns.values
column_names

array(['REC_IR', 'STATNUM', 'ORI', 'ORIADDDT', 'ORINIBRS', 'CITY',
       'STATECOD', 'POPGRP', 'DIVISN', 'REGION', 'AGINDIC', 'CORECTY',
       'COVBYORI', 'FIELDNO', 'JUDDIST', 'NBRSFLG', 'INACTDTE', 'POP1',
       'COUNTY1', 'MSA1', 'LSTPOP1', 'POP2', 'COUNTY2', 'MSA2', 'LSTPOP2',
       'POP3', 'COUNTY3', 'MSA3', 'LSTPOP3', 'POP4', 'COUNTY4', 'MSA4',
       'LSTPOP4', 'POP5', 'COUNTY5', 'MSA5', 'LSTPOP5', 'MASTERYR',
       'QTR1ACT', 'QTR2ACT', 'QTR3ACT', 'QTR4ACT', 'F1QACT', 'F2QACT',
       'F3QACT', 'F4QACT', 'AGNAME', 'CFIPS1', 'CFIPS2', 'CFIPS3',
       'CFIPS4', 'CFIPS5', 'IINCIDNO', 'INCIDDTE', 'DATASRC', 'QUARTER',
       'TNUMVTMS', 'TNUMOFF', 'GOFFRAC', 'OFFCOD1', 'NUMVTM1', 'LOCCOD1',
       'BIASMO1', 'OFFCOD2', 'NUMVTM2', 'LOCCOD2', 'BIASMO2', 'OFFCOD3',
       'NUMVTM3', 'LOCCOD3', 'BIASMO3', 'OFFCOD4', 'NUMVTM4', 'LOCCOD4',
       'BIASMO4', 'OFFCOD5', 'NUMVTM5', 'LOCCOD5', 'BIASMO5', 'OFFCOD6',
       'NUMVTM6', 'LOCCOD6', 'BIASMO6', 'OFFCOD7', 'NUMVTM7', 'LOCCOD7'

#### Many rows' data is shifted one column. The offending data and the entire row that it's in must be shifted left or right. This general purpose does that, but it's slow.

In [7]:
def shift_cell_contents( dataframe, column_name, row_number, direction=None ) :
    """
    shift all rows starting from column_name(inclusive) toward the first column(inclusive), if 
        direction==left, or toward the last column(inclusive) if direction=right.

    WARNING : THIS FUNCTION WILL DISCARD ANY DATA IN THE FIRST/LAST COLUMN.
    """    

    if type(dataframe) != pandas.DataFrame :
        raise TypeError
    
    if type(column_name) != str :
        raise TypeError
    
    if direction==None :
        return   
    if type( direction ) != str :
        raise TypeError

    if direction.lower() != 'left' :
        if direction.lower() != 'right' :
            raise ValueError

    column_number = dataframe.columns.get_loc(column_name)
            
    if direction.lower() == 'left' :
        from_column_number = column_number
        to_column_number = dataframe.shape[1]-1
        range_increment = -1
    elif direction.lower() == 'right' : 
        from_column_number = column_number-1
        to_column_number = 0
        range_increment = 1
       
    for current_column_number in range( from_column_number, to_column_number, range_increment ) :
        try :
            dataframe.iloc[[row_number],[current_column_number]] = dataframe.iloc[
                                                                        [row_number],
                                                                        [current_column_number+range_increment]
                                                                   ]
        except :
            print( 'ERROR in current_column_number ', 
                  current_column_number, ' = range(', from_column_number, ", ", 
                  to_column_number, "; ", range_increment, ')',
                 
                 )
    return dataframe
    

   

#### THIS CODE IS NOT OF GENERAL UTILITY BUT USES "shift_cell_contents(...)" TO CORRECT ERRORS WHERE THE AGNAME COLUMN IS SHIFTED LEFT INTO THE CFIPS1 COLUMN..AND ITS WHOLE ROW WITH IT.

- it's also very slow

In [3]:
agname_column = combined_data_table.columns.get_loc('AGNAME')
cfips1_column = combined_data_table.columns.get_loc('CFIPS1')
total_rows = combined_data_table.shape[0]
beginning_of_filter_slice = 1
end_of_filter_slice =  combined_data_table.shape[0]
row_count = 0


for  CFIPS1_value in combined_data_table['CFIPS1'] :
    if row_count > combined_data_table[beginning_of_filter_slice:end_of_filter_slice].shape[0] :
        row_count += 1
        break
    
    if row_count%100 == 0 :
        print( row_count )
    
    if type(CFIPS1_value) == None :
        row_count += 1        
        continue
        
    if (type(CFIPS1_value ) == int) | (type(CFIPS1_value ) == float) :
        row_count += 1
        continue
        
    try :
        if int(CFIPS1_value) > 0 :
            row_count += 1
            continue
    except :
        pass
            
        
    if (type(CFIPS1_value )) == str :
        if len(CFIPS1_value.strip())==0 :
            row_count += 1
            continue #this cell is effectively empty, move to the next            
        
    # if we're here then we have a string that contains data
    AGNAME_value=combined_data_table.iloc[ row_count, agname_column ].strip()
    if len(AGNAME_value) > 0 :
        combined_data_table.iloc[ row_count, agname_column ]=AGNAME_value + ", " + CFIPS1_value.strip()
        combined_data_table = shift_cell_contents( combined_data_table[beginning_of_filter_slice:end_of_filter_slice],
                                             'CFIPS1', row_count, direction='left' )

NameError: name 'combined_data_table' is not defined

In [9]:
combined_data_table.to_csv(output_path+output_filename, index=False)

#### Now that we finally have clean data we'll extract the series we need.

In [12]:
bias_dataframe = pandas.DataFrame()
bias_data_columns=[ 'CITY', 
                                         'STATECOD', 
                                         'INCIDDTE',
                                         'BIASMO1', 
                                         'BIASMO2', 
                                         'BIASMO3',
                                         'POP1',
                                         'POP2',
                                         'POP3',
                                         'LSTPOP1',
                                         'LSTPOP2',
                                         'LSTPOP3' ]
bias_dataframe = combined_data_table[ bias_data_columns ]

In [13]:
bias_dataframe.head()

,CITY,STATECOD,INCIDDTE,BIASMO1,BIASMO2,BIASMO3,POP1,POP2,POP3,LSTPOP1,LSTPOP2,LSTPOP3
210,RIVIERA,AZ,19990428,12,,,28814,0,0,28733,0,0
211,RIVIERA,AZ,19990505,32,,,28814,0,0,28733,0,0
212,RIVIERA,AZ,19991021,32,,,28814,0,0,28733,0,0
213,RIVIERA,AZ,19991031,15,,,28814,0,0,28733,0,0
214,RIVIERA,AZ,19991103,12,,,28814,0,0,28733,0,0


In [14]:
bias_dataframe.tail()

,CITY,STATECOD,INCIDDTE,BIASMO1,BIASMO2,BIASMO3,POP1,POP2,POP3,LSTPOP1,LSTPOP2,LSTPOP3
5909,BECKLEY,WV,20130523,11,,,17593,0,0,17675,0,0
5910,BECKLEY,WV,20130525,12,,,17593,0,0,17675,0,0
5911,BECKLEY,WV,20130706,41,,,17593,0,0,17675,0,0
5912,BUCKHANNON,WV,20130305,11,,,5649,0,0,5649,0,0
5913,GILLETTE,WY,20130702,32,,,31884,0,0,29816,0,0


In [15]:
bias_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263438 entries, 210 to 5913
Data columns (total 12 columns):
CITY        263438 non-null object
STATECOD    263438 non-null object
INCIDDTE    263438 non-null int64
BIASMO1     263438 non-null int64
BIASMO2     263438 non-null object
BIASMO3     263438 non-null object
POP1        263438 non-null int64
POP2        263438 non-null int64
POP3        263438 non-null int64
LSTPOP1     263438 non-null int64
LSTPOP2     263438 non-null int64
LSTPOP3     263438 non-null int64
dtypes: int64(8), object(4)
memory usage: 26.1+ MB


#### NaN's and None's would definitely mess up the processing, so replace them with empty cells; info() suggests we don't have any, but better safe than sorry.

In [16]:

bias_dataframe=bias_dataframe.fillna( value="" )
bias_dataframe.head()


,CITY,STATECOD,INCIDDTE,BIASMO1,BIASMO2,BIASMO3,POP1,POP2,POP3,LSTPOP1,LSTPOP2,LSTPOP3
210,RIVIERA,AZ,19990428,12,,,28814,0,0,28733,0,0
211,RIVIERA,AZ,19990505,32,,,28814,0,0,28733,0,0
212,RIVIERA,AZ,19991021,32,,,28814,0,0,28733,0,0
213,RIVIERA,AZ,19991031,15,,,28814,0,0,28733,0,0
214,RIVIERA,AZ,19991103,12,,,28814,0,0,28733,0,0


In [17]:
bias_dataframe.tail(20)

,CITY,STATECOD,INCIDDTE,BIASMO1,BIASMO2,BIASMO3,POP1,POP2,POP3,LSTPOP1,LSTPOP2,LSTPOP3
5894,CLARKSBURG,WV,20131204,21,,,34839,0,0,35071,0,0
5895,CHARLESTON,WV,20130124,24,,,50919,0,0,51178,0,0
5896,CHARLESTON,WV,20130827,41,,,50919,0,0,51178,0,0
5897,CHARLESTON,WV,20130828,11,,,50919,0,0,51178,0,0
5898,CHARLESTON,WV,20131101,21,,,50919,0,0,51178,0,0
5899,CHARLESTON,WV,20131115,11,,,50919,0,0,51178,0,0
5900,CHARLESTON,WV,20131117,15,15,,50919,0,0,51178,0,0
5901,CHARLESTON,WV,20131020,21,,,50919,0,0,51178,0,0
5902,DUNBAR,WV,20131029,42,,,7834,0,0,7876,0,0
5903,WELCH,WV,20130624,12,,,14943,0,0,15417,0,0


## Finally ! save the cleaned, intermediate data so we don't have to run this again !

In [18]:
bias_dataframe.to_csv(output_path+output_bias_filename, index=False)